# Modes of Operation Starter

The previous set of challenges showed how AES performs a keyed permutation on a block of data. In practice, we need to encrypt messages much longer than a single block. `A mode of operation` describes how to use a cipher like AES on longer messages.

All modes have serious weaknesses when used incorrectly. The challenges in this category take you to a different section of the website where you can interact with APIs and exploit those weaknesses. Get yourself acquainted with the interface and use it to take your next flag!

Play at https://aes.cryptohack.org/block_cipher_starter

## Challenge overview

### source.py

```python
from Crypto.Cipher import AES


KEY = ?
FLAG = ?


@chal.route('/block_cipher_starter/decrypt/<ciphertext>/')
def decrypt(ciphertext):
    ciphertext = bytes.fromhex(ciphertext)

    cipher = AES.new(KEY, AES.MODE_ECB)
    try:
        decrypted = cipher.decrypt(ciphertext)
    except ValueError as e:
        return {"error": str(e)}

    return {"plaintext": decrypted.hex()}


@chal.route('/block_cipher_starter/encrypt_flag/')
def encrypt_flag():
    cipher = AES.new(KEY, AES.MODE_ECB)
    encrypted = cipher.encrypt(FLAG.encode())

    return {"ciphertext": encrypted.hex()}
```

## Solution

In [4]:
import requests

url = "https://aes.cryptohack.org/block_cipher_starter"

def encrypt_flag():
    return requests.get(f"{url}/encrypt_flag/").json()['ciphertext']

def decrypt(ciphertext):
    plaintext =  requests.get(f"{url}/decrypt/{ciphertext}/").json()['plaintext']
    return bytes.fromhex(plaintext)

enc_flag = encrypt_flag()
flag = decrypt(enc_flag)

print("Flag: " + flag.decode())

Flag: crypto{bl0ck_c1ph3r5_4r3_f457_!}


### Flag: crypto{bl0ck_c1ph3r5_4r3_f457_!}

# Passwords as Keys

It is essential that keys in symmetric-key algorithms are random bytes, instead of passwords or other predictable data. The random bytes should be generated using a cryptographically-secure pseudorandom number generator (CSPRNG). If the keys are predictable in any way, then the security level of the cipher is reduced and it may be possible for an attacker who gets access to the ciphertext to decrypt it.

Just because a key looks like it is formed of random bytes, does not mean that it necessarily is. In this case the key has been derived from a simple password using a hashing function, which makes the ciphertext crackable.

For this challenge you may script your HTTP requests to the endpoints, or alternatively attack the ciphertext offline. Good luck!

Play at https://aes.cryptohack.org/passwords_as_keys

## Challenge overview

### source.py

```python
from Crypto.Cipher import AES
import hashlib
import random


# /usr/share/dict/words from
# https://gist.githubusercontent.com/wchargin/8927565/raw/d9783627c731268fb2935a731a618aa8e95cf465/words
with open("/usr/share/dict/words") as f:
    words = [w.strip() for w in f.readlines()]
keyword = random.choice(words)

KEY = hashlib.md5(keyword.encode()).digest()
FLAG = ?


@chal.route('/passwords_as_keys/decrypt/<ciphertext>/<password_hash>/')
def decrypt(ciphertext, password_hash):
    ciphertext = bytes.fromhex(ciphertext)
    key = bytes.fromhex(password_hash)

    cipher = AES.new(key, AES.MODE_ECB)
    try:
        decrypted = cipher.decrypt(ciphertext)
    except ValueError as e:
        return {"error": str(e)}

    return {"plaintext": decrypted.hex()}


@chal.route('/passwords_as_keys/encrypt_flag/')
def encrypt_flag():
    cipher = AES.new(KEY, AES.MODE_ECB)
    encrypted = cipher.encrypt(FLAG.encode())

    return {"ciphertext": encrypted.hex()}
```

## Solution

In [5]:
import hashlib
import requests
from Crypto.Cipher import AES

def encrypt_flag():
    url = "https://aes.cryptohack.org/passwords_as_keys"
    return bytes.fromhex(requests.get(f"{url}/encrypt_flag/").json()["ciphertext"])

with open("./words", "r") as f:
    words = [w.strip() for w in f.readlines()]
    f.close()

enc_flag = encrypt_flag()
for keyword in words:
    KEY = hashlib.md5(keyword.encode()).digest()
    cipher = AES.new(KEY, AES.MODE_ECB)
    
    flag = cipher.decrypt(enc_flag)
    if b'crypto' in flag:
        print("Flag: " + flag.decode())
        break    

Flag: crypto{k3y5__r__n07__p455w0rdz?}


### Flag: crypto{k3y5__r__n07__p455w0rdz?}